In [ ]:
!pip install tfx==0.30.0

     |████████████████████████████████| 2.4MB 7.3MB/s 
     |████████████████████████████████| 9.6MB 47.7MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 399kB 49.7MB/s 
     |████████████████████████████████| 1.7MB 47.4MB/s 
     |████████████████████████████████| 1.4MB 45.9MB/s 
     |████████████████████████████████| 112kB 56.6MB/s 
     |████████████████████████████████| 394.5MB 39kB/s 
     |████████████████████████████████| 2.9MB 30.8MB/s 
     |████████████████████████████████| 1.3MB 30.1MB/s 
     |████████████████████████████████| 2.3MB 25.3MB/s 
     |████████████████████████████████| 194kB 58.1MB/s 
     |████████████████████████████████| 1.2MB 22.3MB/s 
     |████████████████████████████████| 153kB 56.9MB/s 
     |████████████████████████████████| 1.5MB 30.1MB/s 
     |████████████████████████████████| 17.7MB 210kB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 2.3MB 25.7MB

In [ ]:
!pip install -Uq tensorflow-text

     |████████████████████████████████| 4.3MB 8.1MB/s 
     |████████████████████████████████| 454.3MB 40kB/s 
     |████████████████████████████████| 4.0MB 41.1MB/s 
     |████████████████████████████████| 471kB 57.2MB/s 
ERROR: tfx 0.30.0 has requirement tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.5.*,<3,>=1.15.2, but you'll have tensorflow 2.5.0 which is incompatible.
ERROR: tensorflow-transform 0.30.0 has requirement tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,<2.5,>=1.15.2, but you'll have tensorflow 2.5.0 which is incompatible.


In [ ]:
import glob
import os
import pprint
import re
import tempfile
from shutil import rmtree
from typing import List, Dict, Tuple, Union

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.saved import saved_transform_io
#from tensorflow_transform.tf_metadata import (dataset_metadata, dataset_schema,
#                                             metadata_io, schema_utils)
from tfx.components import (CsvExampleGen,Evaluator, ExampleValidator, 
                            ModelValidator, Pusher, SchemaGen,
                            StatisticsGen, Trainer, Transform)
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import evaluator_pb2, example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

import tensorflow_datasets as tfds
import tensorflow_model_analysis as tfma
import tensorflow_text as text

from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext


In [ ]:

import tensorflow as tf

In [ ]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# location of the pipeline metadata store
_pipeline_root = 'pipeline/'



In [ ]:
base_dir='/content/drive/MyDrive/discord_chatbot/data/'
train_data_dir = base_dir+'train/'
val_data_dir = base_dir+'val/'
test_data_dir = base_dir+'test/'
cnv = 'convert/'

# Data validation

In [ ]:
train_df = pd.read_csv(base_dir+'sentiment_train.csv', skipinitialspace=True)
eval_df = pd.read_csv(base_dir+'sentiment_val.csv', skipinitialspace=True)

In [ ]:
train_stats = tfdv.generate_statistics_from_dataframe(train_df)
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)

In [ ]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'id',BYTES,required,,-
'age',STRING,optional,single,'age'
'sex',STRING,optional,single,'sex'
'keyword',STRING,optional,single,'keyword'
'Column5',STRING,optional,single,'Column5'
'emotion_big',STRING,optional,single,'emotion_big'
'emotion_small',STRING,optional,single,'emotion_small'
'human_1',BYTES,optional,single,-
'system_1',BYTES,optional,single,-


/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'age',"'결단코 아드님도 그 말씀을 들으시고 기뻐하실 거예요.', '너무 힘들고 고생스러웠겠어요. 아프면 사람이 더욱 그리운 법인데 상심하였겠어요.', '노년', '마누라가 돈도 없으면서', '앞으로도 좋은 일만 가득하세요.', '연락은 해보셨어요?', '잔소리를 들어 기분이 언짢으셨군요.', '중년', '청년', '청소년'"
'sex',"'나이 들수록 가족과 친구가 더욱 소중한 법이지. ', '남성', '돈 없고 아픈 사람은 술도 못 먹나?', '여성'"
'keyword',"'가족관계', '건강', '건강, 죽음', '대인관계', '대인관계(부부, 자녀)', '몸이 어디 안 좋으신가요?', '연애, 결혼, 출산', '재정', '재정, 은퇴, 노후준비', '주변 사람들이 참 소중하죠.', '직장, 업무 스트레스', '진로, 취업, 직장', '학교폭력/따돌림', '학업 및 진로'"
'Column5',"'만성질환 무', '만성질환 유', '해당없음'"
'emotion_big',"'기쁨', '기쁨 ', '당황', '분노', '불안', '불안 ', '상처', '슬픔'"
'emotion_small',"'가난한, 불우한', '감사하는', '걱정스러운', '고립된', '괴로워하는', '구역질 나는', '기쁨', '낙담한', '남의 시선을 의식하는', '노여워하는', '눈물이 나는', '느긋', '당혹스러운', '당황', '두려운', '마비된', '만족스러운', '방어적인', '배신당한', '버려진', '부끄러운', '분노', '불안', '비통한', '상처', '성가신', '스트레스 받는', '슬픔', '신뢰하는', '신이 난', '실망한', '악의적인', '안달하는', '안도', '억울한', '열등감', '염세적인', '외로운', '우울한', '자신하는', '조심스러운', '좌절한', '죄책감의', '질투하는', '짜증내는', '초조한', '충격 받은', '취약한', '툴툴대는', '편안한', '한심한', '혐오스러운', '혼란스러운', '환멸을 느끼는', '회의적인', '후회되는', '흥분', '희생된'"


In [ ]:
# Generate evaluation dataset statistics
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# Compare training with evaluation
tfdv.visualize_statistics(
    lhs_statistics=eval_stats, 
    rhs_statistics=train_stats, 
    lhs_name='EVAL_DATASET', 
    rhs_name='TRAIN_DATASET'
)

In [ ]:
# Check evaluation data for errors by validating the evaluation dataset statistics using the reference schema
anomalies =  tfdv.validate_statistics(statistics=eval_stats, schema=schema)

# Visualize anomalies
tfdv.display_anomalies(anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'id',Expected data of type: BYTES but got INT,


In [ ]:
cnv_train = train_df[['emotion_big','system_1']].dropna()
cnv_val = eval_df[['emotion_big','system_1']].dropna()

In [ ]:
cnv_train['emotion_big'] = cnv_train['emotion_big'].str.strip().astype('category')
cnv_val['emotion_big'] = cnv_val['emotion_big'].str.strip().astype('category')

In [ ]:
cnv_train['emotion_big'].dropna()

0        기쁨
1        불안
2        당황
3        기쁨
4        기쁨
         ..
40890    불안
40891    기쁨
40892    슬픔
40893    기쁨
40894    불안
Name: emotion_big, Length: 40877, dtype: category
Categories (6, object): ['기쁨', '당황', '분노', '불안', '상처', '슬픔']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cnv_train['emotion_big'] =le.fit_transform(cnv_train['emotion_big'].str.strip())
print(le.classes_)
cnv_val['emotion_big'] = le.fit_transform(cnv_val['emotion_big'].str.strip())
print(le.classes_)


['기쁨' '당황' '분노' '불안' '상처' '슬픔']
['기쁨' '당황' '분노' '불안' '상처' '슬픔']


In [ ]:
labels = [str(i)+x for i,x in enumerate(le.classes_)]
print(labels)

['0기쁨', '1당황', '2분노', '3불안', '4상처', '5슬픔']


In [ ]:
cnv_train['emotion_big'] = cnv_train['emotion_big'].astype('category')
cnv_val['emotion_big'] = cnv_val['emotion_big'].astype('category')

In [ ]:
cnv_train = cnv_train.rename(columns={'emotion_big':'label'})
cnv_val = cnv_val.rename(columns={'emotion_big':'label'})

In [ ]:
cnv_train

,label,system_1
0,0,아내분이 출산을 하시는군요. 정말 축하드려요.
1,3,약 종류가 많아 번거로우시겠어요.
2,1,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.
3,0,재취업 후 첫 월급이라 정말 기쁘시겠어요.
4,0,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?
...,...,...
40890,3,친구 애완견을 대부분 혼자 돌보시는군요. 떠넘겨질까 봐 불안하시겠어요.
40891,0,검진 결과가 좋게 나와서 안도하겠네요.
40892,5,어머니의 응원이 사라져서 슬프시겠어요. 조금 더 자세히 말해주시겠어요?
40893,0,운동 시설을 쓸 수 있어서 기분이 좋으시겠어요.


In [ ]:
cnv_train.to_csv(train_data_dir+'setiment_train.csv',index=False)
cnv_val.to_csv(val_data_dir+'setiment_val.csv',index=False)

# Interactive context

In [ ]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [ ]:
import glob
import os
import pprint
import re
import tempfile
from shutil import rmtree
from typing import List, Dict, Tuple, Union

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.saved import saved_transform_io
#from tensorflow_transform.tf_metadata import (dataset_metadata, dataset_schema,
#                                             metadata_io, schema_utils)
from tfx.components import (CsvExampleGen,Evaluator, ExampleValidator, 
                            ModelValidator, Pusher, SchemaGen,
                            StatisticsGen, Trainer, Transform)
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import evaluator_pb2, example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

import tensorflow_datasets as tfds
import tensorflow_model_analysis as tfma
import tensorflow_text as text

from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [ ]:
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=45),
                 example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=5)
             ]))
example_gen = CsvExampleGen(input_base=train_data_dir)

In [ ]:
#Test
example_gen = CsvExampleGen(input_base='/content/drive/MyDrive/discord_chatbot/test')

In [ ]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 10
        type_id: 5
        uri: "pipeline/CsvExampleGen/examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:889842,xor_checksum:1625011366,sum_checksum:1625011366"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
try:

    artifact = example_gen.outputs['examples'].get()[0]
    
    print(f'split names: {artifact.split_names}')
    print(f'artifact uri: {artifact.uri}')

except IndexError:
    print("context.run() was no-op")

else:
    artifact_uri = artifact.uri

split names: ["train", "eval"]
artifact uri: pipeline/CsvExampleGen/examples/7


In [ ]:

# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
    

# Run the component
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 11
        type_id: 7
        uri: "pipeline/StatisticsGen/statistics/8"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Plot the statistics generated
context.show(statistics_gen.outputs['statistics'])

In [ ]:

# Instantiate SchemaGen with the output statistics from the StatisticsGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
    
    

# Run the component
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 12
        type_id: 9
        uri: "pipeline/SchemaGen/schema/9"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'A',BYTES,required,,-
'Q',BYTES,required,,-
'label',INT,required,,-


In [ ]:
# Instantiate ExampleValidator with the statistics and schema from the previous steps
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],schema=schema_gen.outputs['schema'])
    
    

# Run the component
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 13
        type_id: 11
        uri: "pipeline/ExampleValidator/anomalies/10"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the output
context.show(example_validator.outputs['anomalies'])

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/discord_chatbot')
# Set up paths.
_transform_constants_module_file = 'tranform_constants.py'

In [ ]:
%%writefile transform.py

import tensorflow as tf
import tensorflow_text as text

from transformers import ElectraTokenizer

MAX_SEQ_LEN = 512  # max number is 512


def preprocessing_fn(inputs):
    """Preprocess input column of text into transformed columns of.
        * input token ids
        * input mask
        * input type ids
    """

    CLS_ID = tf.constant(101, dtype=tf.int64)
    SEP_ID = tf.constant(102, dtype=tf.int64)
    PAD_ID = tf.constant(0, dtype=tf.int64)

    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    
    def tokenize_text(text, sequence_length=MAX_SEQ_LEN):
        """
        Perform the koELECTRA preprocessing from text -> input token ids
        """

        # convert text into token ids
        tokens = tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
        
        # Add start and end token ids to the id sequence
        start_tokens = tf.fill([tf.shape(text)[0], 1], CLS_ID)
        end_tokens = tf.fill([tf.shape(text)[0], 1], SEP_ID)
        tokens = tokens[:, :sequence_length - 2]
        tokens = tf.concat([start_tokens, tokens, end_tokens], axis=1)

        # truncate sequences greater than MAX_SEQ_LEN
        tokens = tokens[:, :sequence_length]

        # pad shorter sequences with the pad token id
        tokens = tokens.to_tensor(default_value=PAD_ID)
        pad = sequence_length - tf.shape(tokens)[1]
        tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)

        # and finally reshape the word token ids to fit the output 
        # data structure of TFT  
        return tf.reshape(tokens, [-1, sequence_length])

    def preprocess_input(text):
        """
        Convert input text into the input_word_ids, input_mask, input_type_ids
        """
        input_word_ids = tokenize_text(text)
        input_mask = tf.cast(input_word_ids > 0, tf.int64)
        input_mask = tf.reshape(input_mask, [-1, MAX_SEQ_LEN])
        
        zeros_dims = tf.stack(tf.shape(input_mask))
        input_type_ids = tf.fill(zeros_dims, 0)
        input_type_ids = tf.cast(input_type_ids, tf.int64)

        return (
            input_word_ids, 
            input_mask,
            input_type_ids
        )

    input_word_ids, input_mask, input_type_ids = \
        preprocess_input(tf.squeeze(inputs['Q'], axis=1))

    return {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids,
        'label': inputs['label']
    }

Overwriting transform.py


In [ ]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

ERROR:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@pipeline/_wheels/tfx_user_code_Transform-0.0+8429b77a59697d11e6ade8b9dedfcaedaf675f2536fce145389457f90dffb235-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'


Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/11/.temp_path/tftransform_tmp/c3ee1e7a9f364250856297f1d0ba7aae/assets


INFO:tensorflow:Assets written to: pipeline/Transform/transform_graph/11/.temp_path/tftransform_tmp/c3ee1e7a9f364250856297f1d0ba7aae/assets


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 14
        type_id: 13
        uri: "pipeline/Transform/transform_graph/11"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 15
        type_id: 5
        uri: "pipeline/Transform/transformed_examples/11"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 16
        type_id: 14
        uri: "pipeline/Transform/updated_analyzer_cache/11"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
from tfx_bsl.coders.example_coder import ExampleToNumpyDict

pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[0].uri

print(train_uri)

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_folders = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
tfrecord_filenames = []
for tfrecord_folder in tfrecord_folders:
    for name in os.listdir(tfrecord_folder):
        tfrecord_filenames.append(os.path.join(tfrecord_folder, name))


# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = ExampleToNumpyDict(serialized_example)
    pp.pprint(example)

pipeline/Transform/transformed_examples/11
{'input_mask': array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_type_ids': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_word_ids': array([  101,  2260, 29997, 30019,   100,   999,   102,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0

In [ ]:
%%writefile trainer.py

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from typing import Text

import absl
import tensorflow as tf
from tensorflow import keras
import tensorflow_transform as tft
from tfx.components.trainer.executor import TrainerFnArgs


LABEL_KEY = 'label'


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def get_model(tf_transform_output, max_seq_length=64):
    
    model = TFElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
 
    model.compile(loss='sparse_categorical_crossentropy', 
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                        metrics=['accuracy']
                        )
    return keras_model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

def _input_fn(file_pattern: Text,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 32) -> tf.data.Dataset:

    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=LABEL_KEY)

    return dataset

# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.

    Args:
      fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 128)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 128)

    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = get_model(tf_transform_output=tf_transform_output)

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps)

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(model,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting trainer.py


In [ ]:

TRAINING_STEPS = 200
EVALUATION_STEPS = 150

trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

ERROR:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 200\n}', 'eval_args': '{\n  "num_steps": 150\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'trainer@pipeline/_wheels/tfx_user_code_Trainer-0.0+631a64fcd60dc3944d4dbdf28f891a7b6d7265a1fa2f11bb137d1a9c6367d545-py3-none-any.whl'} 'run_fn'


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
